# Examples with Dataframes

> **These are Go notebooks**: In order to use the GoNB Jupyter Kernel, please install GoNB from here: [https://github.com/janpfeifer/gonb](https://github.com/janpfeifer/gonb)

Note also that for local package development, you can put: `!*go mod edit -replace "github.com/umbralcalc/stochadex=/path/to/stochadex"` at the top of any cell.

## Simulation inference

WIP - how do we get convenient Poisson data here?

In [ ]:
import (
	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	"github.com/umbralcalc/stochadex/pkg/discrete"

	"github.com/go-echarts/go-echarts/v2/opts"
	"github.com/go-echarts/go-echarts/v2/charts"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

storage := analysis.NewStateTimeStorageFromPartitions(
	[]*simulator.PartitionConfig{{
		Name:              "poisson_data",
		Iteration:         &discrete.PoissonProcessIteration{},
		Params:            simulator.NewParams(map[string][]float64{
		    "rates": {0.002, 0.001, 0.004, 0.001},
		}),
		InitStateValues:   []float64{0.0, 0.0, 0.0, 0.0},
		StateHistoryDepth: 1,
		Seed:              123,
	}},
	&simulator.NumberOfStepsTerminationCondition{
		MaxNumberOfSteps: 100,
	},
	&simulator.ConstantTimestepFunction{
		Stepsize: 1000.0,
	},
	1667980544.0,
)
xRef := analysis.DataRef{PartitionName: "poisson_data", IsTime: true}
yRefs := []analysis.DataRef{{PartitionName: "poisson_data"}}
scatter := analysis.NewScatterPlotFromPartition(storage, xRef, yRefs)
scatter.SetGlobalOptions(charts.WithXAxisOpts(opts.XAxis{Type: "time"}))
gonb_echarts.Display(scatter, "width: 1024px; height:400px; background: white;")

In [ ]:
import (
	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	"github.com/umbralcalc/stochadex/pkg/discrete"

	"gonum.org/v1/gonum/floats"
	"github.com/go-echarts/go-echarts/v2/opts"
	"github.com/go-echarts/go-echarts/v2/charts"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

storage := analysis.NewStateTimeStorageFromPartitions(
	[]*simulator.PartitionConfig{
		{
			Name:              "poisson_data",
			Iteration:         &discrete.PoissonProcessIteration{},
			Params:            simulator.NewParams(map[string][]float64{
				"rates": {0.002, 0.001, 0.004, 0.001},
			}),
			InitStateValues:   []float64{0.0, 0.0, 0.0, 0.0},
			StateHistoryDepth: 1,
			Seed:              123,
		},
		{
			Name:              "differenced_poisson_data",
			Iteration:         &general.ValuesFunctionIteration{
				Function: func(
					params *simulator.Params,
					partitionIndex int,
					stateHistories []*simulator.StateHistory,
					timestepsHistory *simulator.CumulativeTimestepsHistory,
				) []float64 {
					values := make([]float64, stateHistories[partitionIndex].StateWidth)
					floats.SubTo(values, params.Get("data"), stateHistories[int(
						params.GetIndex("partition", 0))].Values.RawRowView(0))
					return values
				},
			},
			Params:            simulator.NewParams(make(map[string][]float64)),
			ParamsAsPartitions: map[string][]string{
				"partition": {"poisson_data"},
			},
			ParamsFromUpstream: map[string]simulator.NamedUpstreamConfig{
				"data": {Upstream: "poisson_data"},
			},
			InitStateValues:   []float64{0.0, 0.0, 0.0, 0.0},
			StateHistoryDepth: 10,
			Seed:              0,
	    },
    },
	&simulator.NumberOfStepsTerminationCondition{
		MaxNumberOfSteps: 100,
	},
	&simulator.ConstantTimestepFunction{
		Stepsize: 1000.0,
	},
	1667980544.0,
)

xRef := analysis.DataRef{PartitionName: "poisson_data", IsTime: true}
yRefs := []analysis.DataRef{{PartitionName: "poisson_data"}}
fillYRefs := []analysis.FillLineRef{{
	Upper: analysis.DataRef{
		PartitionName: "poisson_data",
		Transform: func(values []float64) []float64 { 
			floats.AddConst(3.0, values)
			return values
		},
	},
	Lower: analysis.DataRef{
		PartitionName: "poisson_data",
		Transform: func(values []float64) []float64 { 
			floats.AddConst(-3.0, values)
			return values
		},
	},
}}
line := analysis.NewLinePlotFromPartition(storage, xRef, yRefs, fillYRefs)
line.SetGlobalOptions(charts.WithXAxisOpts(opts.XAxis{Type: "time"}))
gonb_echarts.Display(line, "width: 1024px; height:400px; background: white;")